In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


# Text minig with Quora Insincere Questions Classification


#### Sommaire

I.	ANALYSE EXPLORATOIRE DES DONNEES


II.	PREPARATION DES DONNEES 

1.	Vectorization du texte

    
2.	Selection des variables

III.	MODELISATION 


    
2.	Modèle selectionné

3.	Hyperparameters tuning




In [ ]:
%ls

In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from zipfile import ZipFile
from math import radians, cos, sin, asin, sqrt
from datetime import datetime
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk import wordnet, pos_tag
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords, wordnet as wn
import re
import string

In [ ]:
train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
sub_df_target = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train1 = train.sample(frac=0.3,random_state=200)

In [ ]:
train1.shape

In [ ]:
test.shape

In [ ]:
test1 = test.sample(frac=0.3,random_state=200)

In [ ]:
test1.shape

In [ ]:
import pandas as pd 
import numpy as np
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

#cora_data = pd.read_csv('all/train.csv')
#cora_test_data = pd.read_csv('all/test.csv')
cora_data = train1.copy()
cora_test_data = test1.copy()

# I. ANALYSE EXPLORATOIRE DES DONNEES

In [ ]:
cora_data.info()

In [ ]:
#Verification presence données manquantes
cora_data[cora_data['question_text'].isnull()]

In [ ]:
#Verification presence données manquantes
cora_data[cora_data['target'].isnull()]

Nous avons zero données manquantes dans cette base de données.

In [ ]:
percent_target = cora_data.groupby('target').count()
percent_target['percent'] = 100*(percent_target['question_text']/cora_data['target'].count())
percent_target.reset_index(level=0, inplace=True)
percent_target

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Toxic contents','Positive Questions'
sizes = [6, 94]
explode = (0.1, 0)  # only "explode" the 1st slice (i.e. 'Toxic contents')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.0f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()


### **1. Echantillonnage**

We are in presence of a unbalenced data base. So we will try to increase the rate of the negative comments. (This way i also decrease the size of the database because my personnal computer don't have enought cpu to run the jobs on all the database.)

The final goal is to have 40% of negatives comments.

In [ ]:
cora_data_neg_sample = cora_data[cora_data['target'] == 1] #Negatives comments
cora_data_positive_sample = cora_data[cora_data['target'] == 0].reindex()  #Positive Comments

cora_resampling = pd.concat([pd.DataFrame(cora_data_positive_sample.sample(6000)), #130000
                               pd.DataFrame(cora_data_neg_sample.sample(3650))])

In [ ]:
100*(cora_resampling.groupby('target')['question_text'].count())/cora_resampling['target'].count()

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Toxic contents','Positive Questions'
sizes = [38, 62]
explode = (0.1, 0)  # only "explode" the 1st slice (i.e. 'Toxic contents')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.0f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()


### **2. Features engineering**

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk import wordnet, pos_tag
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords, wordnet as wn
import re
import string

#Cleaning data

def clean_str(chaine):
    chaine = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", chaine)     
    chaine = re.sub(r"\'s", " \'s", chaine) 
    chaine = re.sub(r"\'ve", " \'ve", chaine) 
    chaine = re.sub(r"n\'t", " n\'t", chaine) 
    chaine = re.sub(r"\'re", " \'re", chaine) 
    chaine = re.sub(r"\'d", " \'d", chaine) 
    chaine = re.sub(r"\'ll", " \'ll", chaine) 
    chaine = re.sub(r",", " , ", chaine) 
    chaine = re.sub(r"!", " ! ", chaine) 
    chaine = re.sub(r"\(", " \( ", chaine) 
    chaine = re.sub(r"\)", " \) ", chaine) 
    chaine = re.sub(r"\?", " \? ", chaine) 
    chaine = re.sub(r"\s{2,}", " ", chaine)
    chaine = chaine.lower() #convert all text in lower case
    chaine = chaine.replace(' +', ' ') # Remove double space
    chaine = chaine.strip() # Remove trailing space at the beginning or end
    chaine = chaine.replace('[^a-zA-Z]', ' ' )# Everything not a alphabet character replaced with a space
    #words =  [word for word in chaine.split() if word not in [i for i in string.punctuation]] #Remove punctuations
    words =  [word for word in chaine.split() if word.isalpha()] #droping numbers and punctuations
    return ' '.join(words)

#Tokenization and punctuation removing and stopwords
def tokeniZ_stopWords(chaine):
    chaine = word_tokenize(chaine)
    list_stopWords = set(stopwords.words('english'))
    words = [word for word in chaine if word not in list_stopWords]
    return words

#Stemming 
ps = PorterStemmer()
sb = SnowballStemmer('english')

#Lemmatization
def lemat_words(tokens_list):
    from collections import defaultdict
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lemma_function = WordNetLemmatizer()
    return [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens_list)]
    #for token, tag in pos_tag(tokens_list):
     #   lemma = lemma_function.lemmatize(token, tag_map[tag[0]])

# Define Ngrams function
def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
#Cleaning the data 
cora_resampling['clean_question'] = cora_resampling['question_text'].apply(clean_str)

In [ ]:
#Tokenizing and stopwords removing
cora_resampling['tokeniZ_stopWords_question'] = cora_resampling['clean_question'].apply(tokeniZ_stopWords)

In [ ]:
#Words Stemming
cora_resampling['stemming_question'] = [[ps.stem(word) for word in words] for words in cora_resampling['tokeniZ_stopWords_question'] ]
cora_resampling['stemming_question_for_tfidf'] = [' '.join(words) for words in cora_resampling['stemming_question']] 

In [ ]:
#Words lemmatization
cora_resampling['lemmatize_question'] = cora_resampling['tokeniZ_stopWords_question'].apply(lemat_words)
cora_resampling['lemmatize_question_for_tfidf'] = [' '.join(x) for x in cora_resampling['lemmatize_question'] ]

In [ ]:
#Calcul longueur des commentaires
cora_resampling['question_lenght'] = cora_resampling['question_text'].apply(len)

In [ ]:
#Calcul du nombre de ponctuation par question
from string import punctuation
cora_resampling['number_punctuation'] = cora_resampling['question_text'].apply(
    lambda doc: len([word for word in str(doc) if word in punctuation])) 

In [ ]:
#Number of unique words in the text
cora_resampling['number_of_Unique_words'] = cora_resampling['clean_question'].apply([lambda x : len(set(str(x).split()))])

In [ ]:
#Number of stopwords in the text
list_stopWords = set(stopwords.words('english'))
cora_resampling['number_of_StopWords'] = cora_resampling['clean_question'].apply(
    lambda x : len([w for w in x.lower().split() if w in list_stopWords ]))

In [ ]:
#Number of upper case words
cora_resampling['number_of_uppercase'] = cora_resampling['question_text'].apply(
    lambda x : len([w for w in x.split() if w.isupper()]))

In [ ]:
#Average length of words in the text (whithout stop words)
cora_resampling['average_of_wordsLength'] = cora_resampling['clean_question'].apply(
    lambda x : np.mean([len(w) for w in x.split()]))

In [ ]:
#Number of words in the text
cora_resampling['number_of_words'] = cora_resampling['clean_question'].apply([lambda x : len(str(x).split())])

In [ ]:
cora_resampling.info()

In [ ]:
cora_resampling[['question_lenght', 'number_punctuation', 'number_of_words',
       'number_of_Unique_words', 'number_of_StopWords', 'number_of_uppercase',
       'average_of_wordsLength']].sample(5)

### **3. Data visualisation**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
list_var=['question_lenght', 'number_punctuation', 'number_of_Unique_words', 
          'number_of_StopWords', 'number_of_uppercase', 'average_of_wordsLength']
def var_hist_global(df,X='target',Y=list_var, Title='Features Engineering - Histograms', KDE=False):
    fig, ((ax1, ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3, 2 ,figsize=(14,16))#, sharey=True )
    aX = [ax1, ax2,ax3,ax4,ax5,ax6]
    for i in range(len(list_var)):   
        sns.distplot( df[list_var[i]][df[X]== 1 ].dropna(), label="unsinceres questions" , ax= aX[i], kde= KDE , color = 'red')           
        sns.distplot( df[list_var[i]][df[X]== 0 ].dropna(), label="Sinceres questions"   , ax= aX[i], kde= KDE , color = "olive")
    plt.legend()
    plt.title(Title)
    #plt.show()
    plt.savefig("Features_Engineering_Histograms")
    
var_hist_global(df=cora_resampling,X='target',Y=list_var, Title='Histogramme Quora Questions', KDE=True)

In [ ]:
# Calculate number of obs per group & median to position labels
list_var = ['question_lenght', 'number_of_Unique_words', 'number_of_StopWords']
def violin_boxplott(df,X='target',Y=list_var, Title='Features Engineering - Box plot'): 
    fig, (ax1, ax2 ,ax3) = plt.subplots(1,3 ,figsize=(14,8))#, sharey=True )
    medians = cora_resampling.groupby(['target'])['question_lenght', 'number_of_Unique_words', 'number_of_StopWords'].median().values
 
    sns.boxplot( y=list_var[0],  x=X , data = df, ax= ax1 , palette=['olive','red'])
    sns.boxplot( y=list_var[1],  x=X , data = df, ax= ax2 , palette=['olive','red'])
    sns.boxplot( y=list_var[2],  x=X , data = df, ax= ax3 , palette=['olive','red'])
    #plt.title(Title)
    plt.savefig("Features_Engineering_Boxplot")
violin_boxplott(df=cora_resampling)

On constate que les variables suivantes : 
    - number_punctuation, number_of_uppercase et average_of_wordsLength ne sont pas discriminante par rapport à la cible. 
    En effet, les distribution des questions sincères et pas sincières sont quasiment les memes. 
    Elles seront peu voir pas du tout significatives pour expliquer le modèle.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Code recuperer de : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'quora','br', 'Po', 'th', 'sayi', 'fo', 'Unknown','will','say','now','must','want','much','talks','buy','dont','use','etc','go','ago','lot','ki', 'ba'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.savefig(title)
    plt.tight_layout()  

In [ ]:
#plot_wordcloud(cora_data_neg_sample["question_text"], title="Word Cloud of insincere Questions")
#plot_wordcloud(cora_resampling['tokeniZ_stopWords_question'][cora_resampling['target']== 1]) 
plot_wordcloud(cora_resampling['lemmatize_question_for_tfidf'][cora_resampling['target']== 1], title="Word Cloud of insincere Questions") 

In [ ]:
#plot_wordcloud(cora_data_positive_sample["question_text"], title="Word Cloud of sincere Questions")
plot_wordcloud(cora_resampling['lemmatize_question_for_tfidf'][cora_resampling['target']== 0], title="Word Cloud of sincere Questions")

SI POSSIBLE Essayer de faire l'histogramme du T3gram et Bigram.

In [ ]:
# Code source : https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc
from collections import defaultdict
from wordcloud import  STOPWORDS
import plotly.graph_objs as go
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)

#train1_df = train_df[train_df["target"]==1]
#train0_df = train_df[train_df["target"]==0]
train1_df = cora_resampling[cora_resampling['target']==1]
train0_df = cora_resampling[cora_resampling['target']==0]
## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in train0_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(20), 'blue')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train1_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words of sincere questions", 
                                          "Frequent words of insincere questions"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
py.iplot(fig, filename='word-plots')

#plt.figure(figsize=(10,16))
#sns.barplot(x="ngram_count", y="ngram", data=fd_sorted.loc[:50,:], color="b")
#plt.title("Frequent words for Insincere Questions", fontsize=16)
#plt.show()

In [ ]:
freq_dict = defaultdict(int)
for sent in train0_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'orange')


freq_dict = defaultdict(int)
for sent in train1_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(20), 'orange')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,horizontal_spacing=0.15,
                          subplot_titles=["Frequent bigrams of sincere questions", 
                                          "Frequent bigrams of insincere questions"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Bigram Count Plots - N_gram(2,2)")
py.iplot(fig, filename='word-plots')

In [ ]:
freq_dict = defaultdict(int)
for sent in train0_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'green')


freq_dict = defaultdict(int)
for sent in train1_df["lemmatize_question_for_tfidf"]:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(20), 'green')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04, horizontal_spacing=0.2,
                          subplot_titles=["Frequent trigrams of sincere questions", 
                                          "Frequent trigrams of insincere questions"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Trigram Count Plots - N_gram(3,3)")
py.iplot(fig, filename='word-plots')

# II. PREPARATION DES DONNEES 

In [ ]:
cora_resampling.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_cora_train, X_cora_test, y_cora_train, y_cora_test = train_test_split(
            cora_resampling[['clean_question', 'stemming_question_for_tfidf', 'lemmatize_question_for_tfidf',
                             'tokeniZ_stopWords_question', 'stemming_question', 'lemmatize_question',
                             'question_lenght', 'number_punctuation', 'number_of_StopWords', 'number_of_Unique_words', 'number_of_uppercase','average_of_wordsLength']]
            ,cora_resampling['target'], 
            test_size=0.3, random_state=42)
X_cora_train.shape, X_cora_test.shape, y_cora_train.shape, y_cora_test.shape

### **1. Vectorization des données**

#### **1.1 Tf-Idf Vectorizer** 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(  ngram_range=(1,1), 
                                     analyzer='word',
                                     stop_words='english', 
                                     lowercase=True, 
                                     max_df=0.9, # remove too frequent words
                                     min_df=10, # remove too rare words
                                     max_features = None, # max words in vocabulary, will keep most frequent words
                                     binary=False #If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts.
                                  )

In [ ]:
#Stemmed questions vectorzation
X_tfidf_vectorizer_train = tfidf_vectorizer.fit_transform(X_cora_train['stemming_question_for_tfidf'])
X_tfidf_vectorizer_test = tfidf_vectorizer.transform(X_cora_test['stemming_question_for_tfidf'])

In [ ]:
#Lemmentized questions vectorization
X_tfidf_Lem_vect_train = tfidf_vectorizer.fit_transform(X_cora_train['lemmatize_question_for_tfidf'])
X_tfidf_Lem_vect_test = tfidf_vectorizer.transform(X_cora_test['lemmatize_question_for_tfidf'])

In [ ]:
#bigram questions vectorization
bigram_vectorizer = TfidfVectorizer(  ngram_range=(1,2), 
                                     analyzer='word',
                                     stop_words='english', 
                                     lowercase=True, 
                                     max_df=0.9, # remove too frequent words
                                     min_df=10, # remove too rare words
                                     max_features = None, # max words in vocabulary, will keep most frequent words
                                     binary=False #If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts.
                                  )
X_bigram_vectorizer_train = bigram_vectorizer.fit_transform(X_cora_train['stemming_question_for_tfidf'])
X_bigram_vectorizer_test = bigram_vectorizer.transform(X_cora_test['stemming_question_for_tfidf'])

In [ ]:
#T3gram questions vectorization
t3gram_vectorizer = TfidfVectorizer(  ngram_range=(1,4), 
                                     analyzer='word',
                                     stop_words='english', 
                                     lowercase=True, 
                                     max_df=0.9, # remove too frequent words
                                     min_df=10, # remove too rare words
                                     max_features = None, # max words in vocabulary, will keep most frequent words
                                     binary=False #If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts.
                                  )
X_t3gram_vectorizer_train = t3gram_vectorizer.fit_transform(X_cora_train['stemming_question_for_tfidf'])
X_t3gram_vectorizer_test = t3gram_vectorizer.transform(X_cora_test['stemming_question_for_tfidf'])

In [ ]:
#Range single word to t3gram questions vectorization
st3gram_vectorizer = TfidfVectorizer(  ngram_range=(1,3), 
                                     analyzer='word',
                                     stop_words='english', 
                                     lowercase=True, 
                                     max_df=0.9, # remove too frequent words
                                     min_df=10, # remove too rare words
                                     max_features = None, # max words in vocabulary, will keep most frequent words
                                     binary=False #If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts.
                                  )
X_Singt3gram_vectorizer_train = st3gram_vectorizer.fit_transform(X_cora_train['stemming_question_for_tfidf'])
X_Singt3gram_vectorizer_test  = st3gram_vectorizer.transform(X_cora_test['stemming_question_for_tfidf'])

In [ ]:
X_Singt3gram_vectorizer_train

#### **1.2 Word embedding - Doc2Vec**

In [ ]:
#Word2Vec with preprocessiong questions (without stopwords) 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

d2v_training_data = []
for i, doc in enumerate(X_cora_train['stemming_question']):
    d2v_training_data.append(TaggedDocument(words=doc,tags=[i]))

# ========== learning doc embeddings with doc2vec ==========

# PV stands for 'Paragraph Vector'
# PV-DBOW (distributed bag-of-words) dm=0

d2v = Doc2Vec(d2v_training_data, vector_size=300, window=10, alpha=0.1, min_alpha=1e-4, dm=0, negative=1, epochs=10, min_count=2, workers=4)
d2v_vecs = np.zeros((len(X_cora_train['stemming_question']), 300))
for i in range(len(X_cora_train['stemming_question'])):
    d2v_vecs[i,:] = d2v.docvecs[i]
    
d2v_test = np.zeros((len(X_cora_test['stemming_question']), 300))
for i in range(len(X_cora_test['stemming_question'])):
    d2v_test[i,:] = d2v.infer_vector(X_cora_test['stemming_question'].iloc[i])

In [ ]:
#Word2Vec with lemmatize words
d2v_training_data = []
for i, doc in enumerate(X_cora_train['lemmatize_question']):
    d2v_training_data.append(TaggedDocument(words=doc,tags=[i]))

# ========== learning doc embeddings with doc2vec ==========

# PV stands for 'Paragraph Vector'
# PV-DBOW (distributed bag-of-words) dm=0

d2v = Doc2Vec(d2v_training_data, vector_size=200, window=5, alpha=0.1, min_alpha=1e-4, 
              dm=0, negative=1, epochs=10, min_count=2, workers=4)
d2v_vecs_bigram = np.zeros((len(X_cora_train['lemmatize_question']), 200))
for i in range(len(X_cora_train['lemmatize_question'])):
    d2v_vecs_bigram[i,:] = d2v.docvecs[i]
    
d2v_test_bigram = np.zeros((len(X_cora_test['lemmatize_question']), 200))
for i in range(len(X_cora_test['lemmatize_question'])):
    d2v_test_bigram[i,:] = d2v.infer_vector(X_cora_test['lemmatize_question'].iloc[i])

### **2. FEATURES SELECTION**

Faisons un test non parametrique d'indépendance entre les variables quantitatives créées et la variable cible. 
H0 (indépendance entre une variable données et la varibale cible) est rejetée si la P_value <= 0.05 

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectPercentile
from sklearn.pipeline import Pipeline

In [ ]:
features = SelectKBest(mutual_info_classif,k=2).fit(X_cora_train[['question_lenght', 'number_punctuation', 'number_of_StopWords', 'number_of_Unique_words', 
                                        'number_of_uppercase','average_of_wordsLength']].fillna(0),y_cora_train)
independance_test = np.zeros((6,2))
for idx,i in enumerate(['question_lenght', 'number_punctuation', 'number_of_StopWords', 'number_of_Unique_words', 'number_of_uppercase','average_of_wordsLength']):
    #independance_test[idx,0]= features.pvalues_[idx]
    independance_test[idx,1]= features.scores_[idx]
    #print (i,features.pvalues_[idx],features.scores_[idx])
    #print('%s  %s'%(i,features.scores_[idx]))

In [ ]:
list_var=['question_lenght', 'number_punctuation', 'number_of_StopWords', 'number_of_Unique_words', 'number_of_uppercase','average_of_wordsLength']
independance_df = pd.DataFrame({'Variables': list_var, 'p_values': independance_test[:,0], 'MI': independance_test[:,1]},index=None)
independance_df

On rejette H0 en faveur de l'hypothèse alternative (dépendance avec la variable cible) pour l'ensemble des variables textées.

In [ ]:
plt.figure(figsize=(12, 10))
_ = sns.heatmap(cora_resampling[['question_lenght', 'number_punctuation', 'number_of_StopWords', 
                                 'number_of_Unique_words', 'number_of_uppercase','average_of_wordsLength']].corr()
                ,cmap="YlGnBu", annot=True, fmt=".2f")
plt.savefig("Correlation Matrice")
plt.show()

**Pour les matrices generées à partir des textes via tf_idf et Doc2Vec nous appliquerons un filtre sur les features si nécessaire dans la section qui suit (lors de la modélisation).**

# III. MODELISATION


### **1. Description du process de modélisation**
Nou testerons dans cette section plusieurs modèles de classification tels les SVM, les methodes ensembles (RF, adaboost,...), Reseaux de neurones etc... dans l'objectif de choisir le meilleur modèle. Puis nous optimiserons les hyperparametres des modèles qui nous semblent les plus performants. Enfin nous generons les courbes d'apprentissage afn d'évaluer le niveau d'apprentissage de ces modèles pour verifer l'overfitting où  l'underfitting. 

In [ ]:
from sklearn.metrics import accuracy_score,roc_auc_score, f1_score, balanced_accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedShuffleSplit
from matplotlib import pyplot as plt
plt.style.use('seaborn-whitegrid')

def plot_learning_curve(estimator1, X, y, estimator2, ylim=(0, 1.1), cv=2, n_jobs=-1, 
                        train_sizes=np.linspace(.1, 1.0, 5), scoring=None):
    
    
    plt.figure(figsize=(12,6))
    #plt.title("Learning curves for %s" % type(estimator1).__name__)
    #plt.title("Learning curves for %s" %(estimator1))
    plt.grid()
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.xscale('log')
    train_sizes, train_scores, test_scores = learning_curve(
        estimator1, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,
        scoring=scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.subplot(1, 2, 1)
    plt.title("Learning curves for %s" % type(estimator1).__name__)
    plt.plot(
        train_sizes, train_scores_mean, 'o-',
        color="r", #linewidth=3, 
        label="Training score")
    plt.plot(
        train_sizes, test_scores_mean, 'o-',
        color="olive", 
        label="Cross-validation score")
    plt.fill_between(
        train_sizes, train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std, alpha=0.1,
        color="firebrick")
    plt.fill_between(
        train_sizes, test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std, alpha=0.1, color="darkgoldenrod")
    plt.legend(loc="best")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator2, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,
        scoring=scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.subplot(1, 2, 2)
    plt.title("Learning curves for %s with 70 percent of best features" % type(estimator1).__name__)
    plt.plot(
        train_sizes, train_scores_mean, 'o-',
        color="r", #linewidth=3, 
        label="Training score")
    plt.plot(
        train_sizes, test_scores_mean, 'o-',
        color="olive", 
        label="Cross-validation score")
    plt.fill_between(
        train_sizes, train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std, alpha=0.1,
        color="firebrick")
    plt.fill_between(
        train_sizes, test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std, alpha=0.1, color="darkgoldenrod")
    plt.legend(loc="best")
    plt.savefig("Learning curves for %s" % type(estimator1).__name__)

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

def modelize(list_clf,X,y,X_test,y_test):
    selector = SelectPercentile(mutual_info_classif,percentile=70)
    for clf in list_clf:
        Clf1 = clf().fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf())]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=2 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Training', 'Accuracy':accuracy_score(y,Clf1.predict(X)),'Accuracy with 70% best features':accuracy_score(y,Clf2.predict(X)) })
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Test', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test)),'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        #plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')

In [ ]:
from sklearn.linear_model import LogisticRegressionCV , PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import NuSVC, LinearSVC, SVC, OneClassSVM
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB

In [ ]:
generalized_linear_model = [LogisticRegressionCV,PassiveAggressiveClassifier]
support_vector_machines =  [NuSVC, LinearSVC]   
decisionTreeClassification=[DecisionTreeClassifier]
ensemble_methods = [RandomForestClassifier , ExtraTreesClassifier,AdaBoostClassifier, GradientBoostingClassifier]
naive_bayes_model = [GaussianNB, MultinomialNB, ComplementNB]

#### **1.1 MODELISATION AVEC TF-IDF A PARTIR DES DONNEES SOUMISES A LA STEMMIZATION (1,1)**

In [ ]:
#Generalized Linear Model
modelize(generalized_linear_model,X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

**SVM For Unbalanced problems**

In problems where it is desired to give more importance to certain classes or certain individual samples keywords **class_weight** and **sample_weight** can be used.
**SVC** (but not **NuSVC**) implement a keyword **class_weight** in the **fit** method. 
It’s a dictionary of the form **{class_label : value}**, where value is a floating point number > 0 that sets the parameter **C** of class **class_label** to **C * value**.

In [ ]:
def modelize_svc(list_clf,X,y,X_test,y_test):
    selector = SelectPercentile(mutual_info_classif,percentile=70)
    for clf in list_clf:
        Clf1 = clf(kernel='poly').fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(kernel='poly'))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=2 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Training', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test))
                       ,'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Test', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test))
                       ,'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        #plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
#Support Vector Machine
#modelize_svc([SVC],X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

In [ ]:
#Support Vector Machine
modelize(support_vector_machines,X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

In [ ]:
#Decision Tree
modelize(decisionTreeClassification,X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

In [ ]:
#Ensemble Methods
modelize(ensemble_methods,X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)       

In [ ]:
#Naives bayes
modelize([ MultinomialNB, ComplementNB],X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

In [ ]:
#Neural Network - Consomme beaucoup trop d'energie
Neural_network= [MLPClassifier]
modelize(Neural_network,X_tfidf_vectorizer_train,y_cora_train,X_tfidf_vectorizer_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])

In [ ]:
test_df

In [ ]:
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')

In [ ]:
final_model = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy1', 'Accuracy with 70% best features'])

In [ ]:
#final_model= pd.read_csv('models_stemmisation.csv')
final_model.sort_values(by=['Accuracy1'], ascending=False, axis=0, inplace=True)
final_model.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy':'Accuracy1', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)

In [ ]:
final_model

#### **1.2 MODELISATION AVEC TF-IDF A PARTIR DES DONNEES SOUMISES A LA LEMMENTIZATION (1,1)**

In [ ]:
#Nous utiliserons dans cette étape les modèles preselectionnés dans l'étape précedente.
generalized_linear_model2 = [LogisticRegressionCV,PassiveAggressiveClassifier]
support_vector_machines2 =  [LinearSVC,NuSVC]
ensemble_methods2 = [RandomForestClassifier , ExtraTreesClassifier]
Neural_network= [MLPClassifier]
naive_bayes_model = [MultinomialNB, ComplementNB]

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
modelize(generalized_linear_model,X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
modelize(support_vector_machines,X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
modelize(ensemble_methods,X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
modelize(naive_bayes_model,X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
%%time
modelize(Neural_network,X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
%%time
modelize([DecisionTreeClassifier, SVC],X_tfidf_Lem_vect_train,y_cora_train,X_tfidf_Lem_vect_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy1', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy':'Accuracy1', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisation1_2.csv')
final_model2

### **Conclusion : 
    Pour le modèle LinearSVC on constate que des performance identiques lorsque l'on passe des données vectorizée en stemming au données vectorizée lemmentizées. 
    Pour les autres modèles le lemmentization l'emporte sur les données en stemming.**

#### **1.3 MODELISATION AVEC TF-IDF A PARTIR DE DONNEES STEMMISEES N_GRAMS(1,2)**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
modelize(generalized_linear_model2,X_bigram_vectorizer_train,y_cora_train,X_bigram_vectorizer_test,y_cora_test)

In [ ]:
modelize(support_vector_machines2,X_bigram_vectorizer_train,y_cora_train,X_bigram_vectorizer_test,y_cora_test)

In [ ]:
modelize(ensemble_methods2,X_bigram_vectorizer_train,y_cora_train,X_bigram_vectorizer_test,y_cora_test)

In [ ]:
modelize(naive_bayes_model,X_bigram_vectorizer_train,y_cora_train,X_bigram_vectorizer_test,y_cora_test)

In [ ]:
modelize(Neural_network,X_bigram_vectorizer_train,y_cora_train,X_bigram_vectorizer_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisation1_2.csv')
final_model2

#### **1.4 MODELISATION AVEC TF-IDF A PARTIR DES DONNEES SOUMISES A LA LEMMENTIZATION et TREE_GRAMS (1,3)**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
modelize(generalized_linear_model2,X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test) 

In [ ]:
modelize(support_vector_machines2,X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(ensemble_methods2,X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(naive_bayes_model,X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(Neural_network,X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisaton1_3.csv')
final_model2

#### **1.5 MODELISATION AVEC TF-IDF A PARTIR DES DONNEES SOUMISES A LA STEMMISATION et N_GRAMS (1,4)**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
modelize(generalized_linear_model2,X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(support_vector_machines2,X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(ensemble_methods2,X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(naive_bayes_model,X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
modelize(Neural_network,X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisaton1_4.csv')
final_model2

On constate que la technique de lemmentization donne de meilleurs resultats pour les algorithmes des ensembles methods (random forest, extra Trees, Gradiant Boosting).
Aussi on remarque que les n_gram(1,3) donne de meilleurs resultats que la lemmetization sur les modèle lineaire de type regression logistique et Passive Agressive Classifier. Cette remarque est valable pour le linearSCV.
Pour le MultiLayer Perceptron le resultat est meilleur pour les n_gram(1,3)

#### **2.1 MODELISATION AVEC Doc2Vec A PARTIR DES DONNEES SOUMISES A LA STEMMISATION**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
def modelize(list_clf,X,y,X_test,y_test):
    selector = SelectPercentile(mutual_info_classif,percentile=70)
    for clf in list_clf:
        Clf1 = clf().fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf())]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=2 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Training', 'Accuracy':accuracy_score(y,Clf1.predict(X)),'Accuracy with 70% best features':accuracy_score(y,Clf2.predict(X)) })
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Test', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test)),'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')

In [ ]:
modelize(generalized_linear_model2,d2v_vecs,y_cora_train,d2v_test,y_cora_test) 

In [ ]:
%%time
modelize(support_vector_machines2,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize(ensemble_methods2,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize(Neural_network,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize([DecisionTreeClassifier,AdaBoostClassifier,GradientBoostingClassifier, SVC],d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisatonDo2vec.csv')
final_model2

Add some ML algo from Keras and Tensorflow

#### **2.1 MODELISATION AVEC Doc2Vec A PARTIR DES DONNEES SOUMISES A LA LEMMATISATION**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]

In [ ]:
def modelize(list_clf,X,y,X_test,y_test):
    selector = SelectPercentile(mutual_info_classif,percentile=70)
    for clf in list_clf:
        Clf1 = clf().fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf())]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=2 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Training', 'Accuracy':accuracy_score(y,Clf1.predict(X)),'Accuracy with 70% best features':accuracy_score(y,Clf2.predict(X)) })
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Test', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test)),'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')

In [ ]:
modelize(generalized_linear_model2,d2v_vecs,y_cora_train,d2v_test,y_cora_test) 

In [ ]:
modelize(support_vector_machines2,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize(ensemble_methods2,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize(Neural_network,d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
modelize([DecisionTreeClassifier,AdaBoostClassifier,GradientBoostingClassifier, SVC],d2v_vecs,y_cora_train,d2v_test,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_stemmisatonDo2vec.csv')
final_model2

#### **2.1 MODELISATION AVEC Doc2Vec A PARTIR DES DONNEES SOUMISES A LA LEMMATISATION**

In [ ]:
df_models = pd.DataFrame({'Models':[], 'Sample':[], 'Accuracy':[],'Accuracy with 70% best features':[]})
data=[]
def modelize(list_clf,X,y,X_test,y_test):
    selector = SelectPercentile(mutual_info_classif,percentile=70)
    for clf in list_clf:
        Clf1 = clf().fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf())]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=2 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Training', 'Accuracy':accuracy_score(y,Clf1.predict(X)),'Accuracy with 70% best features':accuracy_score(y,Clf2.predict(X)) })
        data.insert(0,{'Models':type(Clf1).__name__, 'Sample':'Test', 'Accuracy':accuracy_score(y_test,Clf1.predict(X_test)),'Accuracy with 70% best features':accuracy_score(y_test,Clf2.predict(X_test)) })
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')

In [ ]:
modelize(generalized_linear_model2,d2v_vecs_bigram,y_cora_train,d2v_test_bigram,y_cora_test) 

In [ ]:
modelize(support_vector_machines2,d2v_vecs_bigram,y_cora_train,d2v_test_bigram,y_cora_test)

In [ ]:
modelize(ensemble_methods2,d2v_vecs_bigram,y_cora_train,d2v_test_bigram,y_cora_test)

In [ ]:
modelize(Neural_network,d2v_vecs_bigram,y_cora_train,d2v_test_bigram,y_cora_test)

In [ ]:
models_df =pd.concat([pd.DataFrame(data),df_models],ignore_index=True)  
#models_df.sort_values(by=['Accuracy','Models','Sample'], ascending=False, axis=0, inplace=True)
#test_df = pd.DataFrame(np.array(models_df[['Accuracy', 'Accuracy with 70% best features']]),
#                       index=[models_df['Models'],models_df['Sample']], columns=['Accuracy', 'Accuracy with 70% best features'])
test_df1=pd.merge(models_df[models_df['Sample']=='Test'],models_df[models_df['Sample']=='Training'],how='inner',on='Models')
test_df1.sort_values(by=['Accuracy_x'], ascending=False, axis=0, inplace=True)
final_model2 = pd.DataFrame( np.array(test_df1[['Sample_y','Accuracy_y', 'Accuracy with 70% best features_y','Sample_x','Accuracy_x', 'Accuracy with 70% best features_x']]),
            index=test_df1['Models'], columns=['','Accuracy', 'Accuracy with 70% best features','','Accuracy', 'Accuracy with 70% best features'])

final_model2.rename(index=str,columns={'Unnamed: 1':'','Unnamed: 4':'', 'Accuracy.1':'Accuracy', 'Accuracy with 70% best features.1':'Accuracy with 70% best features'},inplace=True)
final_model2.to_csv('models_lemmATISatIonDo2vec.csv')
final_model2

### HYPERPARAMETERS TUNING
Nos meilleurs modèles sont les suivants:
    - LinearSVC appliqué avec une matrice Tf-Idf de données stemmisées en n_grams(1,3)
    - LogisticRegression avec une matrice Tf-Idf de données stemmisées en n_grams(1,4)
    - ExtraTreeClassifier avec une matrice Tf-Idf de données stemmisées en n_grams(1,4)
    - MLPClassifier vec une matrice Tf-Idf de données stemmisées en n_grams(1,4)
    
Nous appliquerons deux méthodes d'optimisation d'hyperparametre afin d'en comparer les resultats : GridSearchCV et HyperOpt    


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, KFold
from sklearn.feature_selection import SelectPercentile
from hyperopt import hp,fmin,Trials, tpe, STATUS_FAIL, STATUS_OK, space_eval, anneal
from hyperopt.pyll import scope
from hyperopt.pyll import stochastic

from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import normalize, StandardScaler

import time
random_state = 42
kf = KFold(n_splits=2,random_state=random_state)
n_iter= 50


**1. ExtraTreeClassifier Hyperparameters Tuning**

In [ ]:
#Parameters Optimization with HyperOpt 
#Tree-structure Parzen Estimator: TPE is a default algorithm for the Hyperopt. It uses Bayesian approach for optimization. 
#At every step it is trying to build probabilistic model of the function and choose the most promising parameters for the next step.
#1. We need to create a function to minimize.
def extraTree_accuracy_cv(params, random_state=random_state, cv=kf, X=X_t3gram_vectorizer_train, y=y_cora_train):
    # the function gets a set of variable parameters in "param"
    # the function gets a set of variable parameters in "param"
    params = {'n_estimators': int(params['n_estimators']),  #The number of trees in the forest.
              'max_features': str(params['max_features']), #The number of features to consider when looking for the best split.
              'min_samples_split': int(params['min_samples_split']), #The minimum number of samples required to split an internal node
              'max_depth': int(params['max_depth'])} #The maximum depth of the tree
    # we use this params to create a new LinearSVC Classifier
    model = ExtraTreesClassifier(random_state=random_state, **params, n_jobs = -1)
    # and then conduct the cross validation with the same folds as before
    try:
        return {'loss' : -cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
                'time' : time.time(),
                'status' : STATUS_OK }
    except (Exception, e):
        return {'status' : STATUS_FAIL,
                'time' : time.time(),
                'exception' : str(e)}

In [ ]:
%%time
# possible values of parameters
space={'n_estimators': hp.quniform('n_estimators', 100, 2000, 1),
       'max_depth' : hp.quniform('max_depth', 2, 80, 1),
       'max_features': hp.choice('max_features', ["sqrt","log2"]),
       'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)  
      }

# trials will contain logging information
trials = Trials()

best=fmin(fn=extraTree_accuracy_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )
# computing the score on the test set
model = ExtraTreesClassifier(random_state=random_state, 
                             n_estimators=int(best['n_estimators']), 
                             max_depth=int(best['max_depth']),
                             min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']),
                             n_jobs=-1)
model.fit(X_t3gram_vectorizer_train,y_cora_train)
tpe_test_score=accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))

In [ ]:
print(stochastic.sample(space))

In [ ]:
print("Best Accuracy score on train set {:.3f} params {}".format( -extraTree_accuracy_cv(space_eval(space,best))['loss'], space_eval(space,best)))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
print("Best Accuracy score on train set {:.3f} params {}".format( -extraTree_accuracy_cv(space_eval(space,best))['loss'], space_eval(space,best)))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
#extraTree_accuracy_cv(best), 
space_eval(space,best)

In [ ]:
tpe_results=np.array([[x['result']['loss'],
                      x['misc']['vals']['max_depth'][0],
                      x['misc']['vals']['n_estimators'][0],
                      x['misc']['vals']['max_features'][0],
                      x['misc']['vals']['min_samples_split'][0]
                      
                      
                      ] for x in trials.trials])

tpe_results_df=pd.DataFrame(tpe_results,
                           columns=['score', 'max_depth', 'n_estimators', 'max_features', 'min_samples_split'])
tpe_results_df.plot(subplots=True,figsize=(10, 10))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(random_state=random_state, n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize([ExtraTreesClassifier],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
#Parameters Optimization with HyperOpt 
#Tree-structure Parzen Estimator: TPE is a default algorithm for the Hyperopt. It uses Bayesian approach for optimization. 
#At every step it is trying to build probabilistic model of the function and choose the most promising parameters for the next step.
#1. We need to create a function to minimize.
def RF_cv(params, random_state=random_state, cv=kf, X=X_t3gram_vectorizer_train, y=y_cora_train):
    # the function gets a set of variable parameters in "param"
    # the function gets a set of variable parameters in "param"
    params = {'n_estimators': int(params['n_estimators']),  #The number of trees in the forest.
              'max_features': str(params['max_features']), #The number of features to consider when looking for the best split.
              'min_samples_split': int(params['min_samples_split']), #The minimum number of samples required to split an internal node
              'max_depth': int(params['max_depth'])} #The maximum depth of the tree
    # we use this params to create a new LinearSVC Classifier
    model = RandomForestClassifier(random_state=random_state, **params, n_jobs = -1)
    # and then conduct the cross validation with the same folds as before
    try:
        return {'loss' : -cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
                'time' : time.time(),
                'status' : STATUS_OK }
    except (Exception, e):
        return {'status' : STATUS_FAIL,
                'time' : time.time(),
                'exception' : str(e)}

In [ ]:
%%time
# possible values of parameters
space={'n_estimators': hp.quniform('n_estimators', 20, 500, 1),
       'max_depth' : hp.quniform('max_depth', 2, 100, 1),
       'max_features': hp.choice('max_features', ["sqrt","log2"]),
       'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)  
      }

# trials will contain logging information
trials = Trials()

best=fmin(fn=RF_cv, # function to optimize
          space=space, 
          algo=anneal.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )
# computing the score on the test set
model = RandomForestClassifier(random_state=random_state, 
                             n_estimators=int(best['n_estimators']),
                             #n_estimators= 302,  
                             max_depth=int(best['max_depth']),
                             min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']),
                             n_jobs=-1)
model.fit(X_t3gram_vectorizer_train,y_cora_train)
tpe_test_score=accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))

In [ ]:
hp.quniform('n_estimators', 20, 500, 50)

In [ ]:
print("Best Accuracy score on train set {:.3f} params {}".format( -RF_cv(space_eval(space,best))['loss'], space_eval(space,best)))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
tpe_results=np.array([[x['result']['loss'],
                      x['misc']['vals']['max_depth'][0],
                      #x['misc']['vals']['n_estimators'][0],
                     # x['misc']['vals']['max_features'][0],
                      x['misc']['vals']['min_samples_split'][0]
                      
                      
                      ] for x in trials.trials])

tpe_results_df=pd.DataFrame(tpe_results,
                           columns=['score', 'max_depth', #'n_estimators',# 'max_features', 
                                    'min_samples_split'])
tpe_results_df.plot(subplots=True,figsize=(10, 10))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        #Clf1 = clf(random_state=random_state, n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
        #                     max_features=str(space_eval(space,best)['max_features']), n_jobs=-1).fit(X,y)
        #Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
        #                     max_features=str(space_eval(space,best)['max_features']), n_jobs=-1))]).fit(X,y)
        Clf1 = clf(random_state=random_state, n_estimators=302, max_depth=96, n_jobs=-1).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=302, max_depth=96,n_jobs=-1) )]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize([RandomForestClassifier],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
from sklearn.model_selection import validation_curve
from matplotlib import pyplot as plt

def plot_validation_curve(estimator, X, y, param_name, param_range,
                          ylim=(0, 1.1), cv=5, n_jobs=-1, scoring=None):
    estimator_name = type(estimator).__name__
    plt.figure(figsize=(10,6))
    plt.title("Validation curves for %s on %s"
              % (param_name, estimator_name))
    plt.grid()
    plt.xlim(min(param_range), max(param_range))
    plt.xlabel(param_name)
    plt.ylabel("Score")
    plt.xscale('log')
    
    train_scores, test_scores = validation_curve(
        estimator, X, y, param_name, param_range,
        cv=cv, n_jobs=n_jobs, scoring=scoring)

    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.plot(
        param_range, train_scores_mean, 'o-',
        color="firebrick", linewidth=3, 
        label="Training score")
    plt.plot(
        param_range, test_scores_mean, 'o-',
        color="darkgoldenrod", 
        label="Cross-validation score")
    plt.fill_between(
        param_range, train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std, alpha=0.1,
        color="firebrick")
    plt.fill_between(
        param_range, test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std, alpha=0.1, color="darkgoldenrod")
    plt.legend(loc="best")



In [ ]:
%%time
%matplotlib inline
cv = StratifiedShuffleSplit(n_splits=10 #nombre de fois qu'on regenere le train test
                            , train_size=.8, random_state=random_state)

clf = ExtraTreesClassifier(n_jobs=-1)
param_name = 'max_depth'
param_range = np.linspace(1,1500,15,dtype=int)
print(param_range)
plot_validation_curve(
    clf, X_t3gram_vectorizer_train, y_cora_train, param_name, param_range, cv=cv.split(X_t3gram_vectorizer_train, y_cora_train), scoring='accuracy')

In [ ]:
param_range

In [ ]:
%%time
%matplotlib inline
cv = StratifiedShuffleSplit(n_splits=1 #nombre de fois qu'on regenere le train test
                            , train_size=.7, random_state=random_state)

clf = ExtraTreesClassifier(n_jobs=-1)
param_name = 'n_estimators'
param_range = np.linspace(1,500,10,dtype=int)
print(param_range)
plot_validation_curve(
    clf, X_t3gram_vectorizer_train, y_cora_train, param_name, param_range, cv=cv.split(X_t3gram_vectorizer_train, y_cora_train), scoring='accuracy')

In [ ]:
%%time

#Parameters Optimization with HyperOpt 
#Tree-structure Parzen Estimator: TPE is a default algorithm for the Hyperopt. It uses Bayesian approach for optimization. 
#At every step it is trying to build probabilistic model of the function and choose the most promising parameters for the next step.
#1. We need to create a function to minimize.
def ExtraTC_cv(params, random_state=random_state, cv=kf, X=X_t3gram_vectorizer_train, y=y_cora_train):
    # the function gets a set of variable parameters in "param"
    # the function gets a set of variable parameters in "param"
    params = {#'n_estimators': int(params['n_estimators']),  #The number of trees in the forest.
              'max_features': str(params['max_features']), #The number of features to consider when looking for the best split.
              'min_samples_split': int(params['min_samples_split']), #The minimum number of samples required to split an internal node
              'max_depth': int(params['max_depth'])} #The maximum depth of the tree
    # we use this params to create a new LinearSVC Classifier
    model = ExtraTreesClassifier(random_state=random_state, **params, n_jobs = -1, n_estimators=110)
    # and then conduct the cross validation with the same folds as before
    try:
        return {'loss' : -cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
                'time' : time.time(),
                'status' : STATUS_OK }
    except (Exception, e):
        return {'status' : STATUS_FAIL,
                'time' : time.time(),
                'exception' : str(e)}
    
# possible values of parameters
space={#'n_estimators': hp.quniform('n_estimators', 80, 180, 1),
       'max_depth' : hp.quniform('max_depth', 215, 430, 1),
       'max_features': hp.choice('max_features', ["sqrt","log2"]),
       'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)  
      }

# trials will contain logging information
trials = Trials()

best=fmin(fn=ExtraTC_cv, # function to optimize
          space=space, 
          algo=anneal.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )
# computing the score on the test set
model = ExtraTreesClassifier(random_state=random_state, 
                             #n_estimators=int(best['n_estimators']),
                             n_estimators= 110,  
                             max_depth=int(best['max_depth']),
                             min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']),
                             n_jobs=-1)
model.fit(X_t3gram_vectorizer_train,y_cora_train)
tpe_test_score=accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))

In [ ]:
print("Best Accuracy score on train set {:.3f} params {}".format( -ExtraTC_cv(space_eval(space,best))['loss'], space_eval(space,best)))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(random_state=random_state, n_estimators=110, max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=110, max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1))]).fit(X,y)
        #Clf1 = clf(random_state=random_state, n_estimators=302, max_depth=96, n_jobs=-1).fit(X,y)
        #Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=302, max_depth=96,n_jobs=-1) )]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize([ExtraTreesClassifier],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
%matplotlib inline
cv = StratifiedShuffleSplit(n_splits=1 #nombre de fois qu'on regenere le train test
                            , train_size=.7, random_state=random_state)

clf = ExtraTreesClassifier(n_jobs=-1)
param_name = 'max_depth'
param_range = np.linspace(1,1500,15,dtype=int)
print(param_range)
plot_validation_curve(
    clf, X_t3gram_vectorizer_train_, y_cora_train, param_name, param_range, cv=cv.split(X_t3gram_vectorizer_train_, y_cora_train), scoring='accuracy')

In [ ]:
%%time
%matplotlib inline
cv = StratifiedShuffleSplit(n_splits=1 #nombre de fois qu'on regenere le train test
                            , train_size=.7, random_state=random_state)

clf = ExtraTreesClassifier(n_jobs=-1)
param_name = 'n_estimators'
param_range = np.linspace(1,700,7,dtype=int)
print(param_range)
plot_validation_curve(
    clf, X_t3gram_vectorizer_train, y_cora_train, param_name, param_range, cv=cv.split(X_t3gram_vectorizer_train, y_cora_train), scoring='accuracy')

In [ ]:
%%time

def ExtraTC_cv(params, random_state=random_state, cv=kf, X=X_t3gram_vectorizer_train, y=y_cora_train):
    # the function gets a set of variable parameters in "param"
    # the function gets a set of variable parameters in "param"
    params = {#'n_estimators': int(params['n_estimators']),  #The number of trees in the forest.
              'max_features': str(params['max_features']), #The number of features to consider when looking for the best split.
              'min_samples_split': int(params['min_samples_split']), #The minimum number of samples required to split an internal node
              'max_depth': int(params['max_depth'])} #The maximum depth of the tree
    # we use this params to create a new LinearSVC Classifier
    model = ExtraTreesClassifier(random_state=random_state, **params, n_jobs = -1, n_estimators=110)
    # and then conduct the cross validation with the same folds as before
    return {'loss' : -cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
                'time' : time.time(),
                'status' : STATUS_OK }

    
# possible values of parameters
space={#'n_estimators': hp.quniform('n_estimators', 80, 180, 1),
       'max_depth' : hp.quniform('max_depth', 400, 500, 1),
       'max_features': hp.choice('max_features', ["sqrt","log2"]),
       'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)  
      }

# trials will contain logging information
trials = Trials()

best=fmin(fn=ExtraTC_cv, # function to optimize
          space=space, 
          algo=anneal.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )
# computing the score on the test set
model = ExtraTreesClassifier(random_state=random_state, 
                             #n_estimators=int(best['n_estimators']),
                             n_estimators= 117,  
                             max_depth=int(best['max_depth']),
                             min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']),
                             n_jobs=-1)
model.fit(X_t3gram_vectorizer_train_,y_cora_train)

In [ ]:
print("Best Accuracy score on train set {:.3f} params {}".format( -ExtraTC_cv(space_eval(space,best))['loss'], space_eval(space,best)))
tpe_test_score=accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))
tpe_test_score=balanced_accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))
print('balanced Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(random_state=random_state, n_estimators=117, max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=110, max_depth=int(best['max_depth']), min_samples_split= int(space_eval(space,best)['min_samples_split']),
                             max_features=str(space_eval(space,best)['max_features']), n_jobs=-1))]).fit(X,y)
        #Clf1 = clf(random_state=random_state, n_estimators=302, max_depth=96, n_jobs=-1).fit(X,y)
        #Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(random_state=random_state, n_estimators=302, max_depth=96,n_jobs=-1) )]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),
                balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),
                balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize([ExtraTreesClassifier],X_t3gram_vectorizer_train_,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
#OPTIMISATION DES HYPERPARAMETRES DU MODEL LINEARSVC AVEC GRIDSEARCH - We want to decrease the classifier's complexty (decrease parameter C of LinearSVC)
from sklearn.model_selection import GridSearchCV
param_grid={'max_depth': np.linspace( 10, 200, 10,dtype=int), # Maximum number of levels in tree
            'n_estimators': np.linspace(100,1000, 10 ,dtype=int), # Number of trees in random forest
            'max_features' : ['auto', 'log2'], # Number of features to consider at every split
            'min_samples_split' : [2, 5, 10], # Minimum number of samples required to split a node
            #'min_samples_leaf': [1, 2, 4], # Minimum number of samples required at each leaf node
           }
model = RandomForestClassifier(random_state=random_state, n_jobs=-1)
kf = KFold(n_splits=3,random_state=random_state)

gs=GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_t3gram_vectorizer_train,y_cora_train)
gs_test_score=accuracy_score(y_cora_test, gs.predict(X_t3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
gs_results_df=pd.DataFrame(np.transpose([gs.cv_results_['mean_test_score'],
                                         gs.cv_results_['param_max_depth'].data,
                                         gs.cv_results_['param_max_features'].data,
                                         gs.cv_results_['param_min_samples_split'].data,
                                         gs.cv_results_['param_n_estimators'].data]),
                                           
                           columns=['score', 'max_depth', 'max_features', 'min_samples_split', 'n_estimators',])
gs_results_df.plot(subplots=True,figsize=(10, 10))


In [ ]:
print((gs.cv_results_).keys())

In [ ]:
gs_results_df.sample(5)

In [ ]:
#gs_results_df.plot([gs_results_df['max_depth'],gs_results_df['score']])
plt.plot(gs_results_df['max_depth'],gs_results_df['score'])

**2. LinearSVC Hyperparameters Tuning**

In [ ]:
%%time
#OPTIMISATION DES HYPERPARAMETRES DU MODEL LINEARSVC AVEC GRIDSEARCH - We want to decrease the classifier's complexty (decrease parameter C of LinearSVC)
from sklearn.model_selection import GridSearchCV
param_grid={'C': np.linspace(0.0000001, 1, 250)}
model = LinearSVC()
kf = KFold(n_splits=2,random_state=42)
n_iter= 50
random_state = 42
gs=GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_Singt3gram_vectorizer_train,y_cora_train)
gs_test_score=accuracy_score(y_cora_test, gs.predict(X_Singt3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
print("Best Accuracy for training{:.3f} params {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy for validation: {:.3f}".format(gs_test_score))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
gs_results_df=pd.DataFrame(np.transpose([gs.cv_results_['mean_test_score'],
                                         gs.cv_results_['param_C'].data]),
                           columns=['score', 'c'])
gs_results_df.plot(subplots=True,figsize=(10, 10))

In [ ]:
%%time
#Let's RECALL THE lINEARsvc with the penality = l1, which results in sparse solutions. Sparse solutions correspond to an implicit feature selection.
from sklearn.model_selection import GridSearchCV
param_grid={'C': np.linspace(0.0000001, 1, 250),
            'penalty' : ['l1'],
            'dual': [False]}
model = LinearSVC()
kf = KFold(n_splits=2,random_state=42)
n_iter= 50
random_state = 42
gs=GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_Singt3gram_vectorizer_train,y_cora_train)
gs_test_score=accuracy_score(y_cora_test, gs.predict(X_Singt3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
gs_results_df=pd.DataFrame(np.transpose([gs.cv_results_['mean_test_score'],
                                         gs.cv_results_['param_C'].data]),
                           columns=['score', 'c'])
gs_results_df.plot(subplots=True,figsize=(10, 10))

**We prefer the hyperparameter in the first case: C = 0.46231161155778894 with best accuracy score = 0.878** 

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_LinearSVC(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C=0.46231161155778894).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=0.46231161155778894))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=42)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize_LinearSVC([LinearSVC],X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
#Nouvel Echantillonage  
cora_data_neg_sample = cora_data[cora_data['target'] == 1] #Negatives comments
cora_data_positive_sample = cora_data[cora_data['target'] == 0].reindex()  #Positive Comments

cora_resampling = pd.concat([pd.DataFrame(cora_data_positive_sample.sample(20000)),
                               pd.DataFrame(cora_data_neg_sample)])
100*(cora_resampling.groupby('target')['question_text'].count())/cora_resampling['target'].count()

In [ ]:
cora_resampling.info()

In [ ]:
#Cleaning the data 
cora_resampling['clean_question'] = cora_resampling['question_text'].apply(clean_str)
#Tokenizing and stopwords removing
cora_resampling['tokeniZ_stopWords_question'] = cora_resampling['clean_question'].apply(tokeniZ_stopWords)
#Words Stemming
cora_resampling['stemming_question'] = [[ps.stem(word) for word in words] for words in cora_resampling['tokeniZ_stopWords_question'] ]
cora_resampling['stemming_question_for_tfidf'] = [' '.join(words) for words in cora_resampling['stemming_question']] 

In [ ]:
cora_resampling.columns

In [ ]:
X_cora_train, X_cora_test, y_cora_train, y_cora_test = train_test_split(
            cora_resampling['stemming_question_for_tfidf']
            ,cora_resampling['target'], 
            test_size=0.3, random_state=42)
X_cora_train.shape, X_cora_test.shape, y_cora_train.shape, y_cora_test.shape

In [ ]:
#Vectorization with tf-idf
X_Singt3gram_vectorizer_train_ = st3gram_vectorizer.fit_transform(X_cora_train_)
X_Singt3gram_vectorizer_test_  = st3gram_vectorizer.transform(X_cora_test_)

In [ ]:
%%time
#OPTIMISATION DES HYPERPARAMETRES DU MODEL LINEARSVC AVEC GRIDSEARCH - We want to increase the regularization of the classifier (decrease parameter C of LinearSVC)
from sklearn.model_selection import GridSearchCV
param_grid={'C': np.linspace(0.0000001, 1, 250)}
model = LinearSVC()
kf = KFold(n_splits=2,random_state=42)
n_iter= 50
random_state = 42
gs=GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_Singt3gram_vectorizer_train_,y_cora_train_)
gs_test_score=accuracy_score(y_cora_test_, gs.predict(X_Singt3gram_vectorizer_test_))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
%%time
#Let's try to increase the data regularization by using the Standardization method
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=60)
def modelize_LinearSVC(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C=0.4056225493975904).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=0.4056225493975904))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='balanced_accuracy')
        print('=============================================')
    
modelize_LinearSVC([LinearSVC],X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
#Let's try to increase the data regularization by using the Standardization method
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=60)
def modelize_LinearSVC(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C=0.4056225493975904).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=0.4056225493975904))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='balanced_accuracy')
        print('=============================================')
norm=StandardScaler(with_mean=False)
modelize_LinearSVC([LinearSVC],norm.fit_transform(X_Singt3gram_vectorizer_train),y_cora_train,norm.transform(X_Singt3gram_vectorizer_test),y_cora_test)

In [ ]:
%%time
#Let's try to increase the data regularization by using the normalization method
modelize_LinearSVC([LinearSVC],normalize(X_Singt3gram_vectorizer_train,norm='l2'),y_cora_train,normalize(X_Singt3gram_vectorizer_test,norm='l2'),y_cora_test)

**3. MLPClassiffer Hyperparameters optimization**

In [ ]:
#Parameters Optimization with HyperOpt 
#1. We need to create a function to minimize.
def MLP_accuracy_cv(params, random_state=random_state, cv=kf, X=X_t3gram_vectorizer_train, y=y_cora_train):
    # the function gets a set of variable parameters in "param" 
    params = {'hidden_layer_sizes': tuple(params['hidden_layer_sizes']), 
              'activation': str(params['activation']),#Activation functions for the hidden layers
              'solver': str(params['solver']), #The solver for weight optimization.
              'alpha': int(params['alpha']), #L2 penalty (regularization term) parameter
              'learning_rate': str(params['learning_rate'])} #Learning rate schedule for weight updates
    # we use this params to create a new LinearSVC Classifier
    model = MLPClassifier(random_state=random_state ,# **params)
                          hidden_layer_sizes = params['hidden_layer_sizes'],
                          activation = params['activation'],
                          solver = params['solver'],
                          alpha = params['alpha'],
                          learning_rate = params['learning_rate'])
    # and then conduct the cross validation with the same folds as before
    try:
        return {'loss' : -cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
                'time' : time.time(),
                'status' : STATUS_OK }
    except (Exception, e):
        return {'status' : STATUS_FAIL,
                'time' : time.time(),
                'exception' : str(e)}

In [ ]:
%%time
# possible values of parameters
space={'hidden_layer_sizes': hp.choice('hidden_layer_sizes' , [(20,10,5,),(5,25,50,),(100,25,5,)]),
       'activation' : hp.choice('activation' , ["identity", "logistic", "tanh", "relu"]), 
       'solver' : hp.choice( 'solver' , ["lbfgs", "sgd", "adam"]),
       'alpha': hp.uniform('alpha',0.0001,0.9),
       'learning_rate': hp.choice('learning_rate' , ["constant", "invscaling", "adaptive"])
      }
      

# trials will contain logging information
trials = Trials()

best=fmin(fn=MLP_accuracy_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state)) # fixing random state for the reproducibility

In [ ]:
%%time
# computing the score on the test set
model = MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] ))
model.fit(X_t3gram_vectorizer_train,y_cora_train)
tpe_test_score=accuracy_score(y_cora_test, model.predict(X_t3gram_vectorizer_test))

In [ ]:
%%time
print("Best Accuracy score on train set {:.3f} params {}".format( -MLP_accuracy_cv(space_eval(space,best))['loss'], space_eval(space,best)))
print('Accuracy score on validation sample {:.3f}'.format(tpe_test_score))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_MLP(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Model = MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] ))
        
        Clf1 = Model.fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] )))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
        
modelize_MLP([MLPClassifier],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
cora_data_neg_sample = cora_data[cora_data['target'] == 1] #Negatives comments
cora_data_positive_sample = cora_data[cora_data['target'] == 0].reindex()  #Positive Comments

cora_resampling = pd.concat([pd.DataFrame(cora_data_positive_sample.sample(4000)),
                               pd.DataFrame(cora_data_neg_sample.sample(2500))])
100*(cora_resampling.groupby('target')['question_text'].count())/cora_resampling['target'].count()

In [ ]:
#Cleaning the data 
cora_resampling['clean_question'] = cora_resampling['question_text'].apply(clean_str)
#Tokenizing and stopwords removing
cora_resampling['tokeniZ_stopWords_question'] = cora_resampling['clean_question'].apply(tokeniZ_stopWords)
#Words lemmatization
cora_resampling['lemmatize_question'] = cora_resampling['tokeniZ_stopWords_question'].apply(lemat_words)
cora_resampling['lemmatize_question_for_tfidf'] = [' '.join(x) for x in cora_resampling['lemmatize_question'] ]
#Vectorization with tf-idf
X_Singt3gram_vectorizer_train = st3gram_vectorizer.fit_transform(X_cora_train['lemmatize_question_for_tfidf'])
X_Singt3gram_vectorizer_test  = st3gram_vectorizer.transform(X_cora_test['lemmatize_question_for_tfidf'])

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_MLP(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Model = MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] ))
        
        Clf1 = Model.fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] )))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=random_state)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='balanced_accuracy')
        print('=============================================')
        
modelize_MLP([MLPClassifier],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

**4. Logistic Regression Hyperparameters tuning**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
random_state = 42

In [ ]:
%%time
#Let's try to increase the model complexity.

param_grid={'C': np.linspace(1, 12, 100),
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga', 'liblinear']}
model = LogisticRegression( n_jobs=-1)
kf = KFold(n_splits=3,random_state=random_state)

gs=GridSearchCV(model, param_grid, scoring='accuracy',  n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_t3gram_vectorizer_train,y_cora_train)
gs_test_score=accuracy_score(y_cora_test, gs.predict(X_t3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
gs_results_df=pd.DataFrame(np.transpose([gs.cv_results_['mean_test_score'],
                                         gs.cv_results_['param_C'].data]),
                           columns=['score', 'c'])
gs_results_df.plot(subplots=True,figsize=(10, 10))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=71)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C=3.3855421686746987).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=3.3855421686746987))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=42)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),
                balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted'),
                balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
    
modelize_LogReg([LogisticRegression],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
#Let's try to increase the model complexity.

param_grid={'C': np.linspace(1, 100, 250), 'penalty': ['l1']}
model = LogisticRegression()
kf = KFold(n_splits=3,random_state=random_state)

gs=GridSearchCV(model, param_grid, scoring='accuracy',  n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_t3gram_vectorizer_train,y_cora_train)
gs_test_score=accuracy_score(y_cora_test, gs.predict(X_t3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
gs_results_df=pd.DataFrame(np.transpose([gs.cv_results_['mean_test_score'],
                                         gs.cv_results_['param_C'].data]),
                           columns=['score', 'c'])
gs_results_df.plot(subplots=True,figsize=(10, 10))

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C= 2.9879518072289155, penalty= 'l1').fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=2.9879518072289155, penalty='l1'))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=42)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),
                balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
norm=StandardScaler(with_mean=False)
modelize_LogReg([LogisticRegression],norm.fit_transform(X_t3gram_vectorizer_train),y_cora_train,norm.transform(X_t3gram_vectorizer_test),y_cora_test)

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C= 3.3855421686746987, penalty= 'l2').fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=3.3855421686746987, penalty='l2'))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=42)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),
                balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
norm=StandardScaler(with_mean=False)
modelize_LogReg([LogisticRegression],norm.fit_transform(X_t3gram_vectorizer_train),y_cora_train,norm.transform(X_t3gram_vectorizer_test),y_cora_test)

In [ ]:
#Nouvel Echantillonage  
cora_data_neg_sample = cora_data[cora_data['target'] == 1] #Negatives comments
cora_data_positive_sample = cora_data[cora_data['target'] == 0].reindex()  #Positive Comments

cora_resampling = pd.concat([pd.DataFrame(cora_data_positive_sample.sample(20000)),
                               pd.DataFrame(cora_data_neg_sample)])
100*(cora_resampling.groupby('target')['question_text'].count())/cora_resampling['target'].count()

#Cleaning the data 
cora_resampling['clean_question'] = cora_resampling['question_text'].apply(clean_str)
#Tokenizing and stopwords removing
cora_resampling['tokeniZ_stopWords_question'] = cora_resampling['clean_question'].apply(tokeniZ_stopWords)
#Words Stemming
cora_resampling['stemming_question'] = [[ps.stem(word) for word in words] for words in cora_resampling['tokeniZ_stopWords_question'] ]
cora_resampling['stemming_question_for_tfidf'] = [' '.join(words) for words in cora_resampling['stemming_question']] 

X_cora_train, X_cora_test, y_cora_train, y_cora_test = train_test_split(
            cora_resampling['stemming_question_for_tfidf']
            ,cora_resampling['target'], 
            test_size=0.3, random_state=42)
X_cora_train.shape, X_cora_test.shape, y_cora_train.shape, y_cora_test.shape

#Vectorization with tf-idf
X_Singt3gram_vectorizer_train = st3gram_vectorizer.fit_transform(X_cora_train)
X_Singt3gram_vectorizer_test  = st3gram_vectorizer.transform(X_cora_test)

X_t3gram_vectorizer_train = t3gram_vectorizer.fit_transform(X_cora_train)
X_t3gram_vectorizer_test  = t3gram_vectorizer.transform(X_cora_test)

In [ ]:
%%time
#OPTIMISATION DES HYPERPARAMETRES DU MODEL LINEARSVC AVEC GRIDSEARCH - We want to increase the regularization of the classifier (decrease parameter C of LinearSVC)
from sklearn.model_selection import GridSearchCV
param_grid={'C': np.linspace(1, 10, 50), 'penalty': ['l2']}
model = LogisticRegression(n_jobs=-1)
kf = KFold(n_splits=3,random_state=random_state)

gs=GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1, cv=kf, verbose=False)
gs.fit(X_t3gram_vectorizer_train,y_cora_train)
gs_test_score=balanced_accuracy_score(y_cora_test, gs.predict(X_t3gram_vectorizer_test))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
print("Best Accuracy on training sample: {:.3f} with hyperparameters {}".format(gs.best_score_, gs.best_params_))

gs_test_scorer=accuracy_score(y_cora_test, gs.predict(X_t3gram_vectorizer_test))
print("Best Accuracy on validation sample: {:.3f} ".format(gs_test_scorer))
print("Best balanced Accuracy on validation sample: {:.3f} ".format(gs_test_score))

In [ ]:
%%time
from sklearn.metrics import balanced_accuracy_score

#Let's try to increase the data regularization by using the Standardization method
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=30)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C=3.272727272727273).fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=3.272727272727273))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='balanced_accuracy')
        print('=============================================')
    
modelize_LogReg([LogisticRegression],X_t3gram_vectorizer_train,y_cora_train,X_t3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
norm=StandardScaler(with_mean=False)
modelize_LogReg([LogisticRegression],norm.fit_transform(X_t3gram_vectorizer_train),y_cora_train,norm.transform(X_t3gram_vectorizer_test),y_cora_test)

In [ ]:
%%time
selector = SelectPercentile(f_classif,percentile=30)
modelize_LogReg([LogisticRegression],normalize(X_t3gram_vectorizer_train),y_cora_train,normalize(X_t3gram_vectorizer_test),y_cora_test)

In [ ]:
%%time
random_state=42
kf = KFold(n_splits=3,random_state=random_state)

searchCV = LogisticRegressionCV(
        Cs=list(np.linspace(0.0001, 100, 250))
        ,penalty='l1'
        ,scoring='accuracy'
        ,cv=kf
        ,random_state=random_state
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='saga'
        ,tol=10
    )
searchCV.fit(X_Singt3gram_vectorizer_train, y_cora_train)

In [ ]:
print ('Max accuracy training sample:', searchCV.scores_[1].mean(axis=0).max())
print('Max accuracy validation sample:', accuracy_score(searchCV.predict(X_Singt3gram_vectorizer_test),y_cora_test))

In [ ]:
%%time
from sklearn.metrics import balanced_accuracy_score

#Let's try to increase the data regularization by using the Standardization method
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf.fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf)]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=51)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Balanced Accuracy Score    F1 Score                Balanced Accuracy Score    F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='balanced_accuracy')
        print('=============================================')
    
modelize_LogReg([LogisticRegressionCV(Cs=list(np.linspace(0.0001, 100, 250)),penalty='l2',scoring='accuracy',cv=kf
     ,random_state=random_state,max_iter=10000,fit_intercept=True,solver='newton-cg',tol=10)],X_Singt3gram_vectorizer_train,y_cora_train,X_Singt3gram_vectorizer_test,y_cora_test)

In [ ]:
%%time
#Lets generate the learning curve of the optimized model
selector = SelectPercentile(f_classif,percentile=70)
def modelize_LogReg(list_clf,X,y,X_test,y_test):
    for clf in list_clf:
        Clf1 = clf(C= 3.3855421686746987, penalty= 'l2').fit(X,y)
        Clf2 = Pipeline([('Feature Selection',selector),('Classification',clf(C=3.3855421686746987, penalty='l2'))]).fit(X,y)
        cv = StratifiedShuffleSplit(n_splits=3 , test_size=.3, random_state=42)
        print('Model : %s' %type(Clf1).__name__)
        print('With all features                                              /    With 70% of the best features')
        print('                 Accuracy Score             F1 Score                Accuracy Score             F1 Score')
        print('training :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y,Clf1.predict(X)),f1_score(y,Clf1.predict(X),average='weighted'),
                balanced_accuracy_score(y,Clf2.predict(X)),f1_score(y,Clf2.predict(X),average='weighted')))
        print('Test     :       %f                   %f           /    %f                   %f' 
              %(balanced_accuracy_score(y_test,Clf1.predict(X_test)),f1_score(y_test,Clf1.predict(X_test),average='weighted')
                ,balanced_accuracy_score(y_test,Clf2.predict(X_test)),f1_score(y_test,Clf2.predict(X_test),average='weighted')))
        plot_learning_curve(Clf1, X, y, Clf2, cv, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring='accuracy')
        print('=============================================')
norm=StandardScaler(with_mean=False)
modelize_LogReg([LogisticRegression],norm.fit_transform(X_t3gram_vectorizer_train),y_cora_train,norm.transform(X_t3gram_vectorizer_test),y_cora_test)

In [ ]:
%%time
random_state=42
kf = KFold(n_splits=3,random_state=random_state)

searchCV = LogisticRegressionCV(
        Cs=list(np.linspace(0.0001, 100, 250))
        ,penalty='l1'
        ,scoring='accuracy'
        ,cv=kf
        ,random_state=random_state
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='saga'
        ,tol=10
    )
searchCV.fit(X_Singt3gram_vectorizer_train, y_cora_train)

In [ ]:
print ('Max accuracy training sample:', searchCV.scores_[1].mean(axis=0).max())
print('Max accuracy validation sample:', accuracy_score(searchCV.predict(X_Singt3gram_vectorizer_test),y_cora_test))

In [ ]:
%%time
random_state=42
kf = KFold(n_splits=3,random_state=random_state)

searchCV = LogisticRegressionCV(
        Cs=list(np.linspace(0.0001, 100, 250))
        ,penalty='l2'
        ,scoring='accuracy'
        ,cv=kf
        ,random_state=random_state
        ,max_iter=10000
        ,fit_intercept=True
        #,solver='saga'
        ,tol=10
    )
searchCV.fit(norm.fit_transform(X_Singt3gram_vectorizer_train), y_cora_train)

In [ ]:
print ('Max accuracy training sample:', searchCV.scores_[1].mean(axis=0).max())
print ('Max accuracy validation sample:', accuracy_score(searchCV.predict(norm.transform(X_Singt3gram_vectorizer_test)),y_cora_test))

In [ ]:
searchCV.get_params

### Conclusion

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, KFold
from sklearn.feature_selection import SelectPercentile
from hyperopt import hp,fmin,Trials, tpe, STATUS_FAIL, STATUS_OK, space_eval, anneal
from hyperopt.pyll import scope
from hyperopt.pyll import stochastic

from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import normalize, StandardScaler

import time
random_state = 42
kf = KFold(n_splits=2,random_state=random_state)
n_iter= 50

In [ ]:
%%time
import time
random_state = 42

kf = KFold(n_splits=2,random_state=random_state)
n_iter= 50
Model_final_MLPClassifier = Pipeline([('Feature Selection',SelectPercentile(f_classif,percentile=70)),('Classification',MLPClassifier(random_state=random_state, activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] )))]).fit(X_t3gram_vectorizer_train_,y_cora_train_)

Model_final_LogReg = Pipeline([('Feature Selection',SelectPercentile(f_classif,percentile=70)),('Classification',LogisticRegression(C=3.272727272727273))]).fit(X_t3gram_vectorizer_train_,y_cora_train_)
Model_final_LinearSVC = Pipeline([('Feature Selection',SelectPercentile(f_classif,percentile=70)),('Classification',LinearSVC(C=0.4056225493975904))]).fit(X_Singt3gram_vectorizer_train_,y_cora_train_)
Model_final_ExtraTreesClassifier = ExtraTreesClassifier(max_depth= 443.0, max_features= 'sqrt', min_samples_split= 5, n_estimators=117).fit(X_t3gram_vectorizer_train_,y_cora_train_)

In [ ]:
#mATRICES DE CONFUSIONS
from sklearn.metrics import confusion_matrix
cm_ETree = confusion_matrix(y_cora_test_, Model_final_ExtraTreesClassifier.predict(X_t3gram_vectorizer_test_))
cm_LinearSVC = confusion_matrix(y_cora_test_, Model_final_LinearSVC.predict(X_Singt3gram_vectorizer_test_))
cm_LogReg = confusion_matrix(y_cora_test_, Model_final_LogReg.predict(X_t3gram_vectorizer_test_))
cm_MLP = confusion_matrix(y_cora_test_, Model_final_MLPClassifier.predict(X_t3gram_vectorizer_test_))

In [ ]:
#MATRICES DE CONFUSIONS NORMALISEES
cm_ExtraTreesClassifier = cm_ETree.astype('float') / cm_ETree.sum(axis=1)[:, np.newaxis]
cm_LinearSVC = cm_LinearSVC.astype('float') / cm_LinearSVC.sum(axis=1)[:, np.newaxis]
cm_LogisticRegression = cm_LogReg.astype('float') / cm_LogReg.sum(axis=1)[:, np.newaxis]
cm_MLPClassifier = cm_MLP.astype('float') / cm_MLP.sum(axis=1)[:, np.newaxis]

In [ ]:
from sklearn.utils.multiclass import unique_labels
classes = unique_labels(y_cora_test_)
#dict_models = {'cm_ExtraTreesClassifier' : 'ExtraTreesClassifier', 'cm_LinearSVC': 'LinearSVC', 'cm_LogisticRegression': 'LogisticRegression', 'cm_MLPClassifier': 'MLPClassifier'}

fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(10,10))
aX = [ax1, ax2,ax3,ax4]
dict_models = ['ExtraTreesClassifier',  'LinearSVC',  'LogisticRegression',  'MLPClassifier']
for IDX, i in enumerate([cm_ExtraTreesClassifier, cm_LinearSVC, cm_LogisticRegression, cm_MLPClassifier]):
    im = aX[IDX].imshow(i, interpolation='nearest', cmap=plt.cm.Blues)
    #aX[IDX].figure.colorbar(im, aX[IDX]=aX[IDX])

    # We want to show all ticks...
    aX[IDX].set(xticks=np.arange(i.shape[1]), yticks=np.arange(i.shape[0]),
           xticklabels=classes, yticklabels=classes, # ... and label them with the respective list entries
           title= dict_models[IDX],
           ylabel='True label',
           xlabel='Predicted label')
    
    # Rotate the tick labels and set their alignment.
    #plt.setp(aX[i].get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fmt = '.2f' if normalize else 'd'
    thresh = i.max() / 2.
    for t in range(i.shape[0]):
        for j in range(i.shape[1]):
            aX[IDX].text(j, t, format(i[t, j], fmt),
                    ha="center", va="center",
                    color="white" if i[t, j] > thresh else "black")

In [ ]:
cm_MLPClassifier

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
fpr1, tpr1, thresholds1 = roc_curve(y_cora_test_, Model_final_MLPClassifier.predict(X_t3gram_vectorizer_test_))
fpr2, tpr2, thresholds2 = roc_curve(y_cora_test_, Model_final_ExtraTreesClassifier.predict(X_t3gram_vectorizer_test_))
fpr3, tpr3, thresholds3 = roc_curve(y_cora_test_, Model_final_LinearSVC.predict(X_Singt3gram_vectorizer_test_))
fpr4, tpr4, thresholds4 = roc_curve(y_cora_test_, Model_final_LogReg.predict(X_t3gram_vectorizer_test_))

print('AUC - MLPClassifier: %.2f ' %(auc(fpr1, tpr1)))
print('AUC - ExtraTreesClassifier: %.2f ' %(auc(fpr2, tpr2)))
print('AUC - LinearSVCr: %.2f ' %(auc(fpr3, tpr3)))
print('AUC - LogisticRegression: %.2f ' %(auc(fpr4, tpr4)))


In [ ]:
plt.figure(figsize=(12,10))
lw = 2
plt.plot(fpr1, tpr1, #color='darkorange',
         lw=lw, label='MLPClassifier')
plt.plot(fpr2, tpr2, #color='bleu', 
         lw=lw, label='ExtratreesClassifier')
plt.plot(fpr3, tpr3, #color='Olive', 
         lw=lw, label='LinearSVC')
plt.plot(fpr4, tpr4, #color='red', 
         lw=lw, label='LogisticRegression')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Proportion mal classée")
plt.ylabel("Proportion bien classée")
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('AUC')


###  Sommission du modèle final

In [ ]:
sub_df = pd.read_csv(zf_test.open('test.csv'))
sub_df_target  = pd.read_csv(zf_test.open('sample_submission.csv'))
#sub_df = pd.read_csv('all/test.csv')
#sub_df_target = pd.read_csv('all/sample_submission.csv')

In [ ]:
sub_df_target.info()

In [ ]:
sub_df.info()

In [ ]:
sub_df.shape

In [ ]:
sub_df.head()

In [ ]:
#Cleaning the data 
sub_df['clean_question'] = sub_df['question_text'].apply(clean_str)

#Tokenizing and stopwords removing
sub_df['tokeniZ_stopWords_question'] = sub_df['clean_question'].apply(tokeniZ_stopWords)

#Words Stemming
sub_df['stemming_question'] = [[ps.stem(word) for word in words] for words in sub_df['tokeniZ_stopWords_question'] ]
sub_df['stemming_question_for_tfidf'] = [' '.join(words) for words in sub_df['stemming_question']] 

In [ ]:
#T3gram questions vectorization
X_t3gram_vect_sub_test  = t3gram_vectorizer.transform(sub_df['stemming_question_for_tfidf'])

In [ ]:
#X_t3gram_vect_sub_test

In [ ]:
Model_final_MLPClassifier = Pipeline([('Feature Selection',SelectPercentile(f_classif,percentile=70)),('Classification',MLPClassifier(random_state=random_state, 
                                                activation = str(space_eval(space,best)['activation'] ),
                                                 solver = str(space_eval(space,best)['solver']),
                                                 alpha = int(best['alpha']),
                                                 hidden_layer_sizes=tuple(space_eval(space,best)['hidden_layer_sizes']),
                                                 learning_rate = str(space_eval(space,best)['learning_rate'] )))]).fit(X_t3gram_vectorizer_train,y_cora_train)

In [ ]:
sub_df['target'] = Model_final_MLPClassifier.predict(X_t3gram_vect_sub_test)

In [ ]:
final_submission1 = pd.merge(sub_df,sub_df_target,how='inner',on='qid' )

In [ ]:
final_submission = (final_submission1[['qid','target']]).rename(columns={'target':'prediction'})

In [ ]:
final_submission.to_csv('final_submission.csv', index=False, sep=',')

In [ ]:
final_submission.head()

In [ ]:
%ls